# Using deep features to build an image classifier

# Fire up GraphLab Create
(See [Getting Started with SFrames](../Week%201/Getting%20Started%20with%20SFrames.ipynb) for setup instructions)

In [1]:
import graphlab

In [2]:
# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 8)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\STRIKE~1\AppData\Local\Temp\graphlab_server_1538180799.log.0


This non-commercial license of GraphLab Create for academic use is assigned to hongng234@gmail.com and will expire on September 06, 2019.


# Load a common image analysis dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set.  

In [3]:
image_train = graphlab.SFrame('image_train_data.csv')
image_test = graphlab.SFrame('image_test_data.csv')

Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_train_data.csv

Parsing completed. Parsed 100 lines in 0.287906 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1943 lines. Lines per second: 3119.55

Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_train_data.csv

...

Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_test_data.csv

Parsing completed. Parsed 100 lines in 0.255575 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_test_data.csv

Parsing completed. Parsed 4000 lines in 1.16959 secs.

# Exploring the image data

In [4]:
graphlab.canvas.set_target('ipynb')

In [5]:
image_train['image'].show()

# Train a classifier on the raw image pixels

We first start by training a classifier on just the raw pixels of the image.

In [6]:
raw_pixel_model = graphlab.logistic_classifier.create(
    image_train,target='label',
    features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1894

...

| 4         | 10       | 1.000000  | 2.417562     | 0.441922          | 0.369369            |

| 5         | 12       | 1.000000  | 2.703375     | 0.450370          | 0.369369            |

| 6         | 13       | 1.000000  | 2.895903     | 0.455121          | 0.378378            |

| 10        | 17       | 1.000000  | 3.595217     | 0.514256          | 0.405405            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Make a prediction with the simple model based on raw pixels

In [7]:
image_test[0:3]['image'].show()

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [8]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [9]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

The model makes wrong predictions for all three images.

# Evaluating raw pixel model on test data

In [10]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.466, 'auc': 0.7185918750000005, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     bird     |       cat       |  117  |
 |  automobile  |       cat       |  107  |
 |     dog      |    automobile   |  210  |
 |     dog      |       dog       |  367  |
 |     cat      |       dog       |  265  |
 |     cat      |    automobile   |  267  |
 |     dog      |       bird      |  228  |
 |  automobile  |       dog       |   71  |
 |  automobile  |       bird      |   89  |
 |     bird     |       bird      |  473  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.45269892284361285, 'log_loss': 1.2241159966774826, 'precision': 0.

The accuracy of this model is poor, getting only about 46% accuracy.

# Can we improve the model using deep features

We only have 2005 data points, so it is not possible to train a deep neural network effectively with so little data.  Instead, we will use transfer learning: using deep features trained on the full ImageNet dataset, we will train a simple model on this small dataset.

In [11]:
len(image_train)

2005

## Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [12]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

As we can see, the column deep_features already contains the pre-computed deep features for this data. 

In [13]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242872, 1.09545, 0.0,0.39363, 0.0, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525088, 0.0, 0.0, 0.0,0.0, 0.0, 9.94829, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566016, 0.0, 0.0, 0.0,0.0, 0.0, 9.9972, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.1298, 0.0, 0.0,0.778194, 0.0, 0.758051, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71787, 0.0, 0.0, 0.0,0.0, 0.0, 9.33936, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57819, 0.0, 0.0, 0.0,0.0, 0.0, 9.00632, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.220678, 0.0,0.0, 0.0, 8.58053, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.237535, 0.0, 0.0,0.0, 0.0, 9.9908, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57379, 0.0, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658936, 0.0, 0.0, 0.0,0.0, 0.0, 9.93748, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features, let's train a classifier

In [14]:
deep_features_model = graphlab.logistic_classifier.create(
    image_train,
    features=['deep_features'],
    target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 4         | 11       | 0.250000  | 2.457364     | 0.772152          | 0.788991            |

| 5         | 12       | 0.250000  | 2.763310     | 0.782700          | 0.788991            |

| 6         | 13       | 0.250000  | 3.047217     | 0.793249          | 0.807339            |

| 10        | 17       | 0.250000  | 4.235564     | 0.863397          | 0.798165            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the deep features model to first few images of test set

In [15]:
image_test[0:3]['image'].show()

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [16]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

The classifier with deep features gets all of these images right!

# Compute test_data accuracy of deep_features_model

As we can see, deep features provide us with significantly better accuracy (about 78%)

In [17]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.783, 'auc': 0.9367594166666653, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |    automobile   |   19  |
 |     cat      |       cat       |  678  |
 |  automobile  |       cat       |   14  |
 |     dog      |       bird      |   53  |
 |     cat      |    automobile   |   39  |
 |     bird     |       cat       |  123  |
 |     cat      |       dog       |  201  |
 |     dog      |       dog       |  706  |
 |     bird     |    automobile   |   30  |
 |  automobile  |    automobile   |  956  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7826063543140489, 'log_loss': 0.6569037588272367, 'precision': 0.7

## Quizzes

### 1. What’s the least common category in the training data?

In [18]:
len(image_train[image_train['label'] == 'bird'])

478

In [19]:
len(image_train[image_train['label'] == 'dog'])

509

In [20]:
len(image_train[image_train['label'] == 'cat'])

509

In [21]:
len(image_train[image_train['label'] == 'automobile'])

509

## 2. Of the images below, which is the nearest ‘cat’ labeled image in the training data to the the first image in the test data (image_test[0:1])?


In [22]:
knn_model = graphlab.nearest_neighbors.create(
    image_train,
    features=['deep_features'],
    label='id')

Starting brute force nearest neighbors model training.

In [23]:
image_test[0:1]['image'].show()

In [24]:
knn_imgs = knn_model.query(image_test[0:1])
knn_imgs

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 17.995ms     |

| Done         |         | 100         | 95.969ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.6237223947,1
0,45646,36.0068799499,2
0,32139,36.5200809583,3
0,25713,36.7548479811,4
0,331,36.8731153276,5


In [25]:
def get_images_from_ids(query_result):
    return image_train.filter_by(
        query_result['reference_label'], 'id')

In [26]:
get_images_from_ids(knn_imgs[0])['image'].show()

### 3. Of the images below, which is the nearest ‘dog’ labeled image in the training data to the the first image in the test data (image_test[0:1])?


In [27]:
image_test[0:1]['image'].show()

In [28]:
knn_imgs = knn_model.query(image_test[0:1], k=10)
knn_imgs.head()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 25.992ms     |

| Done         |         | 100         | 108.965ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.6237223947,1
0,45646,36.0068799499,2
0,32139,36.5200809583,3
0,25713,36.7548479811,4
0,331,36.8731153276,5
0,47131,37.0454005624,6
0,34016,37.0807698982,7
0,45778,37.4025821978,8
0,16604,37.4530513213,9
0,16976,37.464261499,10


In [29]:
knn_imgs_df = get_images_from_ids(knn_imgs)
knn_imgs_df.head()

id,image,label,deep_features,image_array
331,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.510964, 0.0,0.0, 0.0, 11.2724, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
16289,Height: 32 Width: 32,cat,"[0.964288, 0.0, 0.0, 0.0,1.12516, 0.0, 9.31215, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."
16604,Height: 32 Width: 32,cat,"[1.07329, 0.0, 0.0, 0.0,0.0, 0.0, 9.69949, 0.0, ...","[98.0, 99.0, 93.0, 105.0,105.0, 98.0, 118.0, ..."
16976,Height: 32 Width: 32,dog,"[0.755595, 0.0, 0.0, 0.0,0.0, 0.0, 9.46039, ...","[16.0, 17.0, 11.0, 18.0,19.0, 13.0, 20.0, 21.0, ..."
25713,Height: 32 Width: 32,cat,"[0.536971, 0.0, 0.0,0.0894459, 0.236474, ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ..."
32139,Height: 32 Width: 32,cat,"[1.29409, 0.0, 0.0,0.5138, 0.106392, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ..."
34016,Height: 32 Width: 32,cat,"[0.915133, 0.0, 1.08758,0.0, 0.0, 0.0, 10.1857, ...","[146.0, 138.0, 135.0,144.0, 136.0, 133.0, ..."
45646,Height: 32 Width: 32,cat,"[0.983678, 0.0, 0.0, 0.0,0.0, 0.192085, 9.89629, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ..."
45778,Height: 32 Width: 32,cat,"[2.00696, 0.0, 0.318544,0.114534, 0.0, 0.0, ...","[25.0, 21.0, 25.0, 29.0,23.0, 24.0, 28.0, 20.0, ..."
47131,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.643777,1.07462, 1.57788, 0.0, ...","[134.0, 105.0, 66.0,165.0, 120.0, 72.0, ..."


In [30]:
knn_imgs_df[knn_imgs_df['label'] == 'dog'][0:1]['image'].show()

### 4. For the first image in the test data, in what range is the mean distance between this image and its 5 nearest neighbors that were labeled ‘cat’ in the training data?

In [31]:
knn_imgs = knn_model.query(image_test[0:1], k=5)
knn_imgs

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 18.995ms     |

| Done         |         | 100         | 86.818ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.6237223947,1
0,45646,36.0068799499,2
0,32139,36.5200809583,3
0,25713,36.7548479811,4
0,331,36.8731153276,5


In [32]:
def get_images_from_ids(query_result):
    df = image_train.filter_by(
        query_result['reference_label'], 'id')
    df['distance'] = query_result['distance']
    return df

In [33]:
knn_imgs_df = get_images_from_ids(knn_imgs)
knn_imgs_df.head()

id,image,label,deep_features,image_array,distance
331,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.510964, 0.0,0.0, 0.0, 11.2724, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ...",34.6237223947
16289,Height: 32 Width: 32,cat,"[0.964288, 0.0, 0.0, 0.0,1.12516, 0.0, 9.31215, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ...",36.0068799499
25713,Height: 32 Width: 32,cat,"[0.536971, 0.0, 0.0,0.0894459, 0.236474, ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ...",36.5200809583
32139,Height: 32 Width: 32,cat,"[1.29409, 0.0, 0.0,0.5138, 0.106392, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ...",36.7548479811
45646,Height: 32 Width: 32,cat,"[0.983678, 0.0, 0.0, 0.0,0.0, 0.192085, 9.89629, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ...",36.8731153276


In [34]:
knn_imgs_df['distance'].mean()

36.155729322318834

### 5. For the first image in the test data, in what range is the mean distance between this image and its 5 nearest neighbors that were labeled ‘dog’ in the training data?

In [35]:
knn_imgs = knn_model.query(image_test[0:1], k=None)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 16.989ms     |

| Done         |         | 100         | 94.964ms     |

In [36]:
knn_imgs_df = get_images_from_ids(knn_imgs)
knn_imgs_df_dogs = knn_imgs_df[knn_imgs_df['label'] == 'dog']
knn_imgs_df_dogs[0:5]

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array,distance
70,Height: 32 Width: 32,dog,"[1.1298, 0.0, 0.0,0.778194, 0.0, 0.758051, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ...",36.7548479811
107,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.220678, 0.0,0.0, 0.0, 8.58053, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ...",37.0807698982
177,Height: 32 Width: 32,dog,"[0.0, 1.45966, 0.0,0.422992, 0.0, 0.0, ...","[55.0, 75.0, 42.0, 51.0,76.0, 37.0, 57.0, 83.0, ...",37.5227051959
424,Height: 32 Width: 32,dog,"[0.9424, 0.0, 0.220353,0.0, 0.0, 0.0, 10.0445, ...","[60.0, 35.0, 18.0, 63.0,49.0, 38.0, 66.0, 56.0, ...",37.7065587528
462,Height: 32 Width: 32,dog,"[1.43463, 0.0, 0.0, 0.0,0.0, 0.0, 9.27982, 0.0, ...","[86.0, 69.0, 75.0, 57.0,41.0, 48.0, 46.0, 35.0, ...",37.7360002204


In [37]:
knn_imgs_df_dogs[0:5]['distance'].mean()

37.36017640965527